# **Importing Libraries:**

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from keras import callbacks
from scipy.integrate import solve_ivp
from sklearn.metrics import mean_absolute_error

## **Initializing the Variables:**

In [14]:
m_1 = float(input("Enter the mass of 1st body:\n"))
m_2 = float(input("Enter the mass of 2nd body:\n"))

x_1 = np.array(input("Enter the coordinates of 1st body:\n").split(','), float)
x_2 = np.array(input("Enter the coordinates of 2nd body:\n").split(','), float)

v_1 = np.array(input("Enter the velocity vector of 1st body:\n").split(','), float)
v_2 = np.array(input("Enter the velocity vector of 2nd body:\n").split(','), float)

t=float(input("Enter the time at which you want the positions:\n"))


Enter the mass of 1st body:
3456
Enter the mass of 2nd body:
23456789
Enter the coordinates of 1st body:
8765, 43113
Enter the coordinates of 2nd body:
32134, 23552
Enter the velocity vector of 1st body:
34567, 6789
Enter the velocity vector of 2nd body:
876543, 425638
Enter the time at which you want the positions:
134


# **Generating the Training dataset:**

In [15]:
n_train=20000
t_train = np.linspace(0,t,num=n_train)
t_mean_train, t_std_train = np.mean(t_train), np.std(t_train)
t_train_norm = (t_train - t_mean_train) / t_std_train

G=6.67e-11
def final_vel_acc(t, y, m1, m2):
  x1, y1, x2, y2, vx1, vy1, vx2, vy2 = y
  dx=x2-x1
  dy=y2-y1
  r=np.sqrt(dx**2+dy**2)
  ax1=G*m2*dx/(r**3)
  ay1=G*m2*dy/(r**3)
  ax2=-G*m1*dx/(r**3)
  ay2=-G*m1*dy/(r**3)
  return [vx1, vy1, vx2, vy2, ax1, ay1, ax2, ay2]

y_init=[x_1[0], x_1[1], x_2[0], x_2[1], v_1[0], v_1[1], v_2[0], v_2[1]]
sol=solve_ivp(final_vel_acc, [0,t], y_init, t_eval=t_train, args=(m_1, m_2))

positions_train = np.vstack((sol.y[0], sol.y[1], sol.y[2], sol.y[3])).T
pos_mean_train, pos_std_train = np.mean(positions_train, axis=0), np.std(positions_train, axis=0)
positions_train_norm = (positions_train - pos_mean_train) / pos_std_train

x_1_train=positions_train_norm[:,:2]
x_2_train=positions_train_norm[:,2:]


# **Creating the Neural Network**

In [16]:
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
)

model = Sequential()
model.add(Dense(128, activation='tanh', input_shape=(1,)))
model.add(Dense(128, activation='tanh'))
model.add(Dense(128, activation='tanh'))
model.add(Dense(4))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# **Compiling and Training the Network**

In [17]:
opt = Adam(learning_rate=0.001, clipvalue=1.0)
model.compile(optimizer = opt, loss = 'mse', metrics=['mae'])
history = model.fit(t_train_norm, positions_train_norm, batch_size = 32, epochs =500, callbacks=[early_stopping], validation_split=0.2)

Epoch 1/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0348 - mae: 0.0758 - val_loss: 0.0023 - val_mae: 0.0398
Epoch 2/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 2.9212e-04 - mae: 0.0132 - val_loss: 0.0013 - val_mae: 0.0298
Epoch 3/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.5330e-04 - mae: 0.0094 - val_loss: 0.0011 - val_mae: 0.0277
Epoch 4/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 7.6479e-05 - mae: 0.0063 - val_loss: 0.0011 - val_mae: 0.0284
Epoch 5/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2.9813e-05 - mae: 0.0038 - val_loss: 3.9448e-04 - val_mae: 0.0162
Epoch 6/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.2027e-05 - mae: 0.0025 - val_loss: 1.5803e-04 - val_mae: 0.0095
Epoch 7/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.5850e-05 - mae: 0.0027 - val_loss: 2.4862e-04 - val_mae: 0.0125
Epoch 8/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 2.6028e-05 - mae: 0.0035 - val_loss: 1.3057e-04 - val_mae: 0.0098
Epoc

# **Generating the test dataset:**

In [18]:
n_test=2000
t_test = np.linspace(0,t,num=n_test)
t_mean_test, t_std_test = np.mean(t_test), np.std(t_test)
t_test_norm = (t_test - t_mean_test) / t_std_test

sol=solve_ivp(final_vel_acc, [0,t], y_init, t_eval=t_test, args=(m_1, m_2))

positions_test = np.vstack((sol.y[0], sol.y[1], sol.y[2], sol.y[3])).T
pos_mean_test, pos_std_test = np.mean(positions_test, axis=0), np.std(positions_test, axis=0)
positions_test_norm = (positions_test - pos_mean_test) / pos_std_test

x_1_test=positions_test_norm[:,0:2]
x_2_test=positions_test_norm[:,2:4]


# **Running the model on testing dataset:**

In [19]:
y_pred_test=model.predict(t_test_norm)
x_1_pred_test=y_pred_test[:,0:2]
x_2_pred_test=y_pred_test[:,2:4]
mae_x_1 = mean_absolute_error(x_1_test, x_1_pred_test)
mae_x_2 = mean_absolute_error(x_2_test, x_2_pred_test)
print("Mean absolute error for body 1 and body 2 :\n", mae_x_1 , mae_x_2)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Mean absolute error for body 1 and body 2 :
 0.004973098739283912 0.004503959492256297


# **Final Prediction**

In [20]:
t_input = np.array([t])
t_input_norm = (t_input - t_mean_train) / t_std_train
y_pred_norm = model.predict(t_input_norm)
y_pred = y_pred_norm * pos_std_train + pos_mean_train

x_1_pred = y_pred[0][:2]
x_2_pred = y_pred[0][2:]

print("Position of body 1:", x_1_pred)
print("Position of body 2:", x_2_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
Position of body 1: [4609434.35959631  949478.71447008]
Position of body 2: [1.16485219e+08 5.65204824e+07]
